## Problem

ある工場では２つの製品Ｉ、II を作っている。これらの製品 1kg を作るのに必要となる
原料、得られる利益（万円／kg）および原料の１日あたりの最大使用量(kg)は下表に示す
とおりである。これらの条件をもとに、１日の利益を最大にする問題をＬＰで定式化せよ。

|      | 製品1 | 製品2 | 使用可能量 |
|    - |     - |     - |          - |
|    E |     1 |     2 |          8 |
|    F |     3 |     0 |         12 |
|    G |     0 |     4 |         12 |
| 利益 |     4 |     6 |            |

(https://www.fit.ac.jp/~song/lecture/06/LP0606.pdf)

In [1]:
import pulp
import pandas as pd

In [2]:
df = pd.DataFrame(
    columns=["製品1", "製品2", "使用可能量"],
    index=["E", "F", "G", "利益"],
    data=[
        [1, 2, 8],
        [3, 0, 12],
        [0, 4, 12],
        [4, 6]
         ]
    )
df

,製品1,製品2,使用可能量
E,1,2,8.0
F,3,0,12.0
G,0,4,12.0
利益,4,6,NaN


In [3]:
problem = pulp.LpProblem(name="product", sense=pulp.LpMaximize)
problem

product:
MAXIMIZE
None
VARIABLES

In [4]:
products = df.columns[:-1].values.tolist()
materials = df.index[:-1].values.tolist()
print("products:", products)
print("materials:", materials)

products: ['製品1', '製品2']
materials: ['E', 'F', 'G']


In [5]:
table = pulp.LpVariable.dicts("", (products), cat=pulp.LpInteger)
table

{'製品1': _製品1, '製品2': _製品2}

In [6]:
problem += pulp.lpSum([table[p] * df[p]["利益"] for p in table.keys()])
# df["製品1"]["利益"]
problem

product:
MAXIMIZE
4*_製品1 + 6*_製品2 + 0
VARIABLES
_製品1 free Integer
_製品2 free Integer

In [7]:
for m in materials:
    problem += pulp.lpSum([table[p] * df[p][m] for p in table.keys()]) <= df["使用可能量"][m]
problem

product:
MAXIMIZE
4*_製品1 + 6*_製品2 + 0
SUBJECT TO
_C1: _製品1 + 2 _製品2 <= 8

_C2: 3 _製品1 <= 12

_C3: 4 _製品2 <= 12

VARIABLES
_製品1 free Integer
_製品2 free Integer

In [8]:
status = problem.solve(pulp.PULP_CBC_CMD(msg=0))
print("Status", pulp.LpStatus[status])

Status Optimal


In [9]:
print("=" * 3, "Result", "=" * 29)
print('利益: {}'.format(pulp.value(problem.objective)))
print("-" * 40)
print("生産量内訳")
for p in products:
    print(table[p].value())
print("=" * 40)

=== Result =============================
利益: 28.0
----------------------------------------
生産量内訳
4.0
2.0
